In [98]:
import pandas as pd
import numpy as np

In [289]:
data=pd.read_csv('data/raw/sales_train_validation.csv')

In [100]:
data=pd.read_csv('data/raw/sales_train_validation.csv')
calendar=pd.read_csv('data/raw/calendar.csv')
prices=pd.read_csv('data/raw/sell_prices.csv')

# EDA

In [101]:
data.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1


In [102]:
for col in data.columns.to_list()[0:6]:
    print('there are {} different values for the {} column'.format(data[col].nunique(),col))

there are 30490 different values for the id column
there are 3049 different values for the item_id column
there are 7 different values for the dept_id column
there are 3 different values for the cat_id column
there are 10 different values for the store_id column
there are 3 different values for the state_id column


In [103]:
prices.tail(3)

,store_id,item_id,wm_yr_wk,sell_price
6841118,WI_3,FOODS_3_827,11619,1.0
6841119,WI_3,FOODS_3_827,11620,1.0
6841120,WI_3,FOODS_3_827,11621,1.0


In [104]:
prices.wm_yr_wk.min()

11101

In [105]:
prices.wm_yr_wk.nunique()

282

In [106]:
prices[prices['item_id']=='HOBBIES_1_001'].wm_yr_wk.min()

11325

# Feature design

## weekly sales

In [107]:
new_calendar=calendar.copy()

In [108]:
for x in calendar.index:
    '''redoing to number of the weeks to turn the calculations easier'''
    new_calendar.wm_yr_wk.iloc[x]=(int(x/7))

<ipython-input-108-1718fc9b1ddf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_calendar.wm_yr_wk.iloc[x]=(int(x/7))


In [109]:
week_maping=pd.Series(new_calendar.wm_yr_wk.values, index=calendar.wm_yr_wk).to_dict() #creating a mapping of the weeks to be further used

In [267]:
weekly_sales = pd.DataFrame(data['id'])
week_columns = [week for week in range(273)]

# Use the `reindex()` method to add columns for each week
weekly_sales = weekly_sales.reindex(columns=['id'] + week_columns)


In [271]:
for week in range(273):
    mean = data.iloc[:,6+7*week: 13+7*week].mean(axis=1)
    weekly_sales[week] = mean
    weekly_sales.loc[data[data.iloc[:,6:13*week].mean(axis=1)==0].index, week] = np.nan
weekly_sales[0].replace(0,np.nan,inplace=True)

In [274]:
weekly_sales.to_csv('data/intermediate/weekly_sales.csv')

In [288]:
weekly_sales=pd.read_csv('data/intermediate/weekly_sales.csv')

for col in weekly_sales.columns[2:]:
    new_col_name = int(col)
    weekly_sales = weekly_sales.rename(columns={col: new_col_name})

## Adjusting sales according to decreases/increases on prices.

For example, a study by McKinsey & Company found that a 1% decrease in price can result in a 0.25% to 0.75% increase in sales for consumer goods such as food, beverages, and household products. Another study by the National Bureau of Economic Research found that the price elasticity of demand for beer was approximately -0.8, indicating that a 10% decrease in price could lead to an 8% increase in sales.

The principle of price elasticity of demand still applies for price increases, meaning that as prices increase, demand tends to decrease. However, the extent to which demand decreases in response to a price increase varies depending on the product and the market. Generally, the price elasticity of demand tends to be higher for products that have readily available substitutes or for which consumers have a high sensitivity to price changes. A study by the National Bureau of Economic Research found that the price elasticity of demand for beer was approximately -0.8, indicating that a 10% increase in price could lead to an 8% decrease in sales. However, this number will vary depending on the product and market conditions.

In [232]:
prices['price_ma_8'] = prices.groupby('item_id')['sell_price'].rolling(window=8, min_periods=1).mean().reset_index(0, drop=True)
prices['inc-dec'] = prices['sell_price']/prices['price_ma_8']
prices['id'] = prices['item_id'] +'_' + prices['store_id'] + '_validation'
prices.to_csv('data/processed/sell_prices.csv')


In [233]:
data_to_adjust_inc = prices[(prices['inc-dec']>1.1)]
data_to_adjust_inc = data_to_adjust_inc [data_to_adjust_inc['wm_yr_wk']<11613]

data_to_adjust_dec = prices[(prices['inc-dec']<0.9)]
data_to_adjust_dec = data_to_adjust_dec [data_to_adjust_dec['wm_yr_wk']<11613]

In [316]:
# Create a dictionary to store the adjusted values for each item and store
adjusted_values = {}

# Iterate over each row in data_to_adjust_inc
for _, row in data_to_adjust_inc.iterrows():
    id = row['id']
    week_nr=row['wm_yr_wk']
    
    # Calculate the adjusted value
    adjusted_value = 1 - ((row['inc-dec'] - 1)*0.8)
    
    # Retrieve the original sales data for the item and store
    sales_data = weekly_sales[(weekly_sales['id'] == id)].loc[:,week_maping.get(week_nr)]
    
    # Calculate the adjusted sales data and store it in the dictionary
    adjusted_sales_data = sales_data*adjusted_value
    adjusted_values[(id, week_nr)] = adjusted_sales_data.values[0]

for _, row in data_to_adjust_dec.iterrows():
    id = row['id']
    week_nr=row['wm_yr_wk']
    
    # Calculate the adjusted value
    adjusted_value = 1+(1-row['inc-dec'])*0.8
    
    # Retrieve the original sales data for the item and store
    sales_data = weekly_sales[(weekly_sales['id'] == id)].loc[:,week_maping.get(week_nr)]
    
    # Calculate the adjusted sales data and store it in the dictionary
    adjusted_sales_data = sales_data*adjusted_value
    adjusted_values[(id, week_nr)] = adjusted_sales_data.values[0]    

In [317]:
for (id,week_nr), adjusted_sales_data in adjusted_values.items():
    weekly_sales.loc[(weekly_sales['id'] == id), week_maping.get(week_nr)] = adjusted_sales_data

In [320]:
weekly_sales.to_csv('data/processed/weekly_sales.csv')

## monthly sales

In [325]:
monthly_sales=pd.DataFrame(weekly_sales['id'])

for month in range(68):  
    monthly_sales[month]=np.nan  
    monthly_sales[month]=7*(weekly_sales[month*4]+weekly_sales[month*4+1]+weekly_sales[month*4+2]+weekly_sales[month*4+3])
monthly_sales[69]=weekly_sales[272]*7

In [324]:
monthly_sales

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,69
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.0,18.0,16.0,12.00000,21.0,26.000000,19.000000,30.0,28.000000,2.25
1,NaN,NaN,NaN,0.000000,0.0,7.0,3.0,11.0,4.000000,9.000000,...,15.0,9.0,9.0,11.00000,27.0,4.000000,4.000000,7.0,5.000000,0.25
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,21.0,13.0,20.00000,42.0,13.000000,9.000000,14.0,9.000000,1.75
3,NaN,10.0,21.0,33.000000,25.0,45.0,44.0,50.0,54.000000,46.000000,...,52.0,50.0,58.0,46.00000,45.0,49.000000,64.000000,49.0,56.000000,3.50
4,NaN,NaN,NaN,0.000000,50.0,14.0,16.0,38.0,56.945056,23.268577,...,39.0,38.0,38.0,36.00000,45.0,15.000000,32.000000,30.0,34.000000,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,77.54166,112.0,104.0,57.133438,33.0,25.0,19.0,20.0,26.000000,22.000000,...,17.0,18.0,15.0,21.86143,27.0,21.686752,24.472832,9.0,4.000000,0.25
30486,29.00000,40.0,24.0,25.000000,11.0,25.0,29.0,19.0,15.000000,21.000000,...,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.0,6.804177,0.00
30487,129.00000,87.0,37.0,31.000000,28.0,39.0,13.0,0.0,14.000000,33.000000,...,13.0,5.0,3.0,8.00000,23.0,23.000000,19.000000,34.0,25.000000,1.00
30488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.0,47.0,43.0,58.00000,41.0,41.000000,34.000000,22.0,30.000000,1.25


In [326]:
monthly_sales.to_csv('data/processed/monthly_sales.csv')